# A/B-тестирование

**Оглавление**
1. [Детализация задачи](#task)
2. [Предобработка данных](#preprocessing)
3. [Оценка корректности проведения теста](#corr)
4. [Исследовательский анализ данных](#analysis)  
5. [Оценка результатов A/B-тестирования](#ABtest)
6. [Общие выводы](#summary)

<a id="task"></a> 
## Детализация задачи
**Цель проекта:** провести оценку результатов A/B-теста `recommender_system_test`.

**Задачи**
- Оценить корректность проведения теста
- Проанализировать результаты теста

### Предобработка данных
- Обработать дубликаты и описать их природу
- Обработать пропущенные значения и описать их природу
- Проверить корректность заполнения столбцов
- Объединить таблицы
- Преобразовать типы где это требуется

### Оценка корректности проведения теста
- Проверить данные на требования технического задания. 
- Проверить время проведения теста на пересечение с маркетинговыми активностями. 
- Оценить аудиторию теста на пересечение с другими тестами и на появление в 2 группах одновременно. 
- Проверить равномерность распределения по тестовым группам и правильность их формирования.

### Исследовательский анализ данных
- Количество событий на пользователя одинаково распределены в выборках?
- Как число событий в выборках распределено по дням?
- Как меняется конверсия в воронке в выборках на разных этапах?
- Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

### Оценка результатов A/B-тестирования
- Вычисление метрик из Техзадания и сравнение их с требуемым результатом
- Проверка статистической разницы долей z-критерием.
    
### Общие выводы
- Выводы по этапу исследовательского анализа данных
- Оценка результатов A/B-тестирования
- Общее заключение о корректности проведения теста.

**Техническое задание**

- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

<a id="preprocessing"></a> 
## Предобработка данных

In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats as st
import math as mth

In [70]:
# прочитаем файлы с данными и взглянем на них
try:
    data = pd.read_csv('final_ab_events.csv')
    marketing = pd.read_csv('ab_project_marketing_events.csv')
    all_users = pd.read_csv('final_ab_new_users.csv')
    ab_users = pd.read_csv('final_ab_participants.csv')

except Exception as e1:
    try:
        data = pd.read_csv('/datasets/final_ab_events.csv')
        marketing = pd.read_csv('/datasets/ab_project_marketing_events.csv')
        all_users = pd.read_csv('/datasets/final_ab_new_users.csv')
        ab_users = pd.read_csv('/datasets/final_ab_participants.csv')
    except Exception as e2:
        print('Error:', e1, e2)

display(data.head())
display(marketing.head())
display(all_users.head())
ab_users.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [71]:
data.info()
print()
marketing.info()
print()
all_users.info()
print()
ab_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id  

In [72]:
# пропуски обнаружены только в таблице с событиями в столбце details
print(data[data['details'].isna()]['event_name'].unique())
print(data[~data['details'].isna()]['event_name'].unique())

['product_cart' 'product_page' 'login']
['purchase']


Мы видим, что в столбце details заполнены все данные с событием purchase, а по всем остальным событиям стоят пропуски

In [73]:
print('Дубликатов в таблице data:', data.duplicated().sum(), data.duplicated().mean())
print('Дубликатов в таблице marketing:', marketing.duplicated().sum(), marketing.duplicated().mean())
print('Дубликатов в таблице all_users:', all_users.duplicated().sum(), all_users.duplicated().mean())
print('Дубликатов в таблице ab_users:', ab_users.duplicated().sum(), ab_users.duplicated().mean())

Дубликатов в таблице data: 0 0.0
Дубликатов в таблице marketing: 0 0.0
Дубликатов в таблице all_users: 0 0.0
Дубликатов в таблице ab_users: 0 0.0


Дубликатов ни в одной таблице не обнаружено

In [74]:
# осталось преобразовать столбцы в формат даты, сократив часы
data['event_dt'] = pd.to_datetime(pd.to_datetime(data['event_dt']).dt.date)
all_users['first_date'] = pd.to_datetime(all_users['first_date'])
marketing['start_dt'] = pd.to_datetime(marketing['start_dt'])
marketing['finish_dt'] = pd.to_datetime(marketing['finish_dt'])

<a id="corr"></a> 
## Оценка корректности проведения теста

In [75]:
# для начала проверим, нет ли в таблице со всеми пользователями повторяющихся user_id
all_users['user_id'].nunique(),  len(all_users['user_id'])

(61733, 61733)

In [76]:
# теперь добавим в таблицу с распределением пользователей данные о них
ab_users = ab_users.merge(all_users, on='user_id', how='left')
# и посмотрим, сколько у нас тестов и сколько уникальных пользователей
print(ab_users['ab_test'].value_counts())
print('Уникальных пользователей в ab_users:', ab_users['user_id'].nunique())

interface_eu_test          11567
recommender_system_test     6701
Name: ab_test, dtype: int64
Уникальных пользователей в ab_users: 16666


Всего проводится два AB теста, но мы видим, что уникальных пользователей меньше чем размер таблицы. Выходит, что 1602 пользователя либо участвуют сразу в обоих тестах, либо попали сразу в две группы.

In [77]:
# проверим пользователей на попадание сразу в 2 группы по какому-либо тесту
((ab_users.query('ab_test == "recommender_system_test"').groupby('user_id')['user_id'].count() > 1).sum(), 
(ab_users.query('ab_test == "interface_eu_test"').groupby('user_id')['user_id'].count() > 1).sum())

(0, 0)

Получается, в данных 1602 пользователя, которые участвуют сразу в двух тестах. Нам подойдут пользователи, которые  находятся в группе А не интересующего нас теста "interface_eu_test", но если они находятся в группе В в нем, это исказит результаты.

In [78]:
# для начала выделим всех 1602 пользователей из двух тестов
abnormal_users = ab_users.groupby('user_id', as_index=False).agg(num_of_entr=('user_id', 'count'))
abnormal_users.query('num_of_entr > 1', inplace=True)
# оставим только тех, которые во втором тесте находятся в группе В
abnormal_users = abnormal_users[abnormal_users['user_id']
                 .isin(ab_users.query('ab_test == "interface_eu_test" and group ==  "B"')['user_id'])]
abnormal_users = abnormal_users['user_id']
len(abnormal_users)

783

In [79]:
# проверим, что даты регистрации пользователей из нашего теста соотвествуют ТЗ 
(ab_users.query('ab_test == "recommender_system_test"')['first_date'].min(), 
ab_users.query('ab_test == "recommender_system_test"')['first_date'].max())

(Timestamp('2020-12-07 00:00:00'), Timestamp('2020-12-21 00:00:00'))

Дата регистрации пользователей соответсвтвует ТЗ

In [80]:
print(ab_users.query('ab_test == "recommender_system_test"')['region'].value_counts())
print('Доля клиентов в тесте по региону EU:', '{:.2%}'.format(
      len(ab_users.query('ab_test == "recommender_system_test" and region == "EU"')) /
      len(all_users.query('region == "EU" and first_date <= "2020-12-21" and first_date >= "2020-12-07"'))))

EU           6351
N.America     223
APAC           72
CIS            55
Name: region, dtype: int64
Доля клиентов в тесте по региону EU: 15.00%


Доля пользователей, выбранных по региону EU соотвествует ТЗ, при этом их количество соответствует ожидаемым. Однако в тест попало 350 пользователей из других регионов. Их тоже добавим в список ненормальных. 

In [81]:
abnormal_users = pd.concat([abnormal_users, 
                 ab_users.query('ab_test == "recommender_system_test" and region != "EU"')['user_id']]).drop_duplicates()
len(abnormal_users)

1133

Итого мы имеем 1133 пользователей из 6701, добавленных некорректно. Исключим их из списка, а также уберем пользователей из второго теста.

In [82]:
# проверим как соотносятся группы А и В в полученном списке до и после исключения некорретных пользователей
print('До исключения некорректных пользователей:\n', 
      ab_users[ab_users['ab_test']  == "recommender_system_test"]['group'].value_counts())
print()
ab_users = ab_users[~ab_users['user_id'].isin(abnormal_users) & (ab_users['ab_test']  == "recommender_system_test")]
print('После исключения некорректных пользователей:\n', ab_users['group'].value_counts())

До исключения некорректных пользователей:
 A    3824
B    2877
Name: group, dtype: int64

После исключения некорректных пользователей:
 A    3195
B    2373
Name: group, dtype: int64


Распределение по группам произошло явно с перекосом в сторону группы А, пользователей в этой группе почти в полтора раза больше, чем в группе В.

In [83]:
# проверим, какие маркетинговые активности проходили в период исследования
marketing[marketing['regions'].str.contains('EU') & (marketing['finish_dt'] >= '2020-12-07')]

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03


К сожалению, для любого пользователя условие увеличения метрик за 14 дней пересекается с рожденственским и новогодним промо. С другой стороны, смысл АВ исследования и состоит в том, чтобы исключить сезонные искажения при исследовании.

<a id="analysis"></a>
## Исследовательский анализ данных

In [84]:
# распишем воронку пользователя
funnel_sec = ["login", "product_page", "product_cart", "purchase"]
# добавим данные о пользователях в таблицу с действиями
data = data.merge(ab_users[['user_id', 'group', 'first_date', 'device']], on='user_id', how='left').dropna(subset=['group'])
# согласно ТЗ нас не будут интересовать события, произошедшие спустя 14 дней после регистрации пользователя, удалим их
data = data[data['event_dt'] - data['first_date'] <= pd.Timedelta(days=14)]
data.head()

,user_id,event_dt,event_name,details,group,first_date,device
5,831887FE7F2D6CBA,2020-12-07,purchase,4.99,A,2020-12-07,Android
17,3C5DD0288AC4FE23,2020-12-07,purchase,4.99,A,2020-12-07,PC
58,49EA242586C87836,2020-12-07,purchase,99.99,B,2020-12-07,iPhone
71,2B06EB547B7AAD08,2020-12-07,purchase,4.99,A,2020-12-07,PC
74,A640F31CAC7823A6,2020-12-07,purchase,4.99,B,2020-12-07,PC


In [85]:
# проверим аномальные значения по количеству событий на каждого пользователя
data.groupby(['user_id', 'group', 'event_name'], as_index=False).agg(event_quantity=('event_dt', 'count'))\
        .sort_values(by='event_quantity', ascending=False).head(10)

,user_id,group,event_name,event_quantity
428,115EBC1CA027854A,B,product_page,8
427,115EBC1CA027854A,B,product_cart,8
426,115EBC1CA027854A,B,login,8
3609,89545C7F903DBA34,B,login,7
4435,A7A055BA12053CBA,A,product_page,7
4434,A7A055BA12053CBA,A,login,7
2933,6E3DF9C69A6B607E,A,login,7
5632,D182471A77C0DC13,B,product_page,7
2934,6E3DF9C69A6B607E,A,product_cart,7
6581,F8185432C71D445E,A,login,7


In [86]:
# аномальной активности не выявлено, проверим еще нет ли пользователя, пропустившего первый этап воронки
data.pivot_table('event_dt', 'user_id', 'event_name', 'count').fillna(0).query('login == 0')

event_name,login,product_cart,product_page,purchase
user_id,,,,
5FF8B6AB257B404F,0.0,0.0,0.0,1.0


In [87]:
# этого странного пользователя стоит удалить из таблицы
data = data[~data['user_id'].isin(data.pivot_table('event_dt', 'user_id', 'event_name', 'count')
                              .fillna(0).query('login == 0').index)]

In [88]:
# посмотрим как среднее количество событий на пользователя распределено по группам
fig = px.bar((data.groupby(['user_id', 'group', 'event_name'], as_index=False).agg(event_quantity=('event_dt', 'count'))
                 .groupby(['group', 'event_name'], as_index=False)['event_quantity'].mean()), 
             x='event_name', y='event_quantity',
             color='group', barmode='group', text_auto='.2f',
             category_orders={"event_name": funnel_sec})

fig.update_layout(width=700, height=300, 
                 title='Среднее количество событий на пользователя, дошедшего до события',
                 xaxis_title='Событие',  yaxis_title='Количество событий')
fig.show()

In [89]:
# так же можно посмотреть количество среднее количество событий по отношению ко всем пользователям группы
mean_events = data.groupby(['group', 'event_name'], as_index=False).agg(event_quantity=('event_dt', 'count')) 
mean_events.loc[mean_events['group'] == 'A', 'event_quantity'] =\
    mean_events.loc[mean_events['group'] == 'A', 'event_quantity'] / data[data['group'] == 'A']['user_id'].nunique()
mean_events.loc[mean_events['group'] == 'B', 'event_quantity'] =\
    mean_events.loc[mean_events['group'] == 'B', 'event_quantity'] / data[data['group'] == 'B']['user_id'].nunique()

fig = px.bar(mean_events, 
             x='event_name', y='event_quantity',
             color='group', barmode='group', text_auto='.2f',
             category_orders={"event_name": funnel_sec})

fig.update_layout(width=700, height=300, 
                 title='Среднее количество событий на всех пользователей из группы',
                 xaxis_title='Событие',  yaxis_title='Количество событий')
fig.show()

Мы видим, что пользователи, дошедшие до каждого этапа по группе А ведут себя активнее пользователей группы В, первые совершают примерно 3 действия на любом этапе, а вторые - только 2.5. При этом если смотреть среднее количество на всех пользователей из группы, то на одного пользователя из группы А в среднем приходится 3 логина, 2 просмотра страницы с товарами и по одной покупке, тогда как для пользователей из группы В - 2.6, 1.4 и 0.7 соотвественно. Можно сказать, что пользователи из группы А в целом ведут себя ощутимо активнее.

In [90]:
# теперь проверим как события распределены по времени в разрезе групп
fig = px.line(data.groupby(['group', 'event_dt'], as_index=False).agg(event_quantity=('user_id', 'count')), 
             x='event_dt', y='event_quantity', color='group')

fig.update_layout(width=600, height=300, 
                 title='Количество событий в день в разрезе групп',
                 xaxis_title='Дата',  yaxis_title='Количество событий')
fig.show()

In [91]:
# и как события распределены по времени в разрезе типов
fig = px.line(data.groupby(['event_dt', 'event_name'], as_index=False).agg(event_quantity=('user_id', 'count')), 
             x='event_dt', y='event_quantity', color='event_name', category_orders={"event_name": funnel_sec})

fig.update_layout(width=670, height=300, 
                 title='Количество событий в день в разрезе типов',
                 xaxis_title='Дата',  yaxis_title='Количество событий')
fig.show()

Если рассматривать динамику событий по дням, то, во-впервых, стоит отметить, что в данные попали события только по 29.12.2020, при том, что ТЗ обозначало конец эксперимента как 04.01.2021, а из динамики данных никак не следует, что после 29.12 действия могли полностью прекартится. Поэтому стоит отметить, что данные по требуемой конверсии за 14 дней будут несколько занижены для пользователей, зарегистировавшихся позже 15.12.2020. Так же мы можем наблюдать сильный всплеск активности с середины декабря и пиком 21.12, причем в группе А этот всплеск значительно выше. В целом можно отметить, что в первые несколько дней эксперимента активности групп были примерно одинаковыми (с учетом разого количества пользователей в группах), но потом группа В стала все больше отставать и, начиная с 14, декабря, пользователи группы В совершали в 4-5 раз меньше активных действий чем в группе А.

In [92]:
# посотрим на активных пользователей
fig = px.line(data.groupby(['event_dt', 'group'], as_index=False).agg(users=('user_id', 'nunique')), 
             x='event_dt', y='users', color='group')

fig.update_layout(width=600, height=300, 
                 title='Количество активных пользователей',
                 xaxis_title='Дата',  yaxis_title='Количество пользователей')
fig.show()

На этом графике мы видим ожидаемое - активных пользователй из группы В после 13.12 стало в разы меньше, чем в группе А. Сезонный всплеск с 14.12 нивелировал падение интереса в группе В и общая активность за следующую неделю там немного выросла, однако мы не видим аналогичного группе А роста.

In [93]:
# посмотрим на конверсию по дням
conversion_days = data.pivot_table('user_id', ['group','event_dt'], 'event_name', 'nunique').fillna(0).reset_index()
conversion_days['pr_p_conv'] = conversion_days['product_page'] / conversion_days['login']
conversion_days['pr_c_conv'] = conversion_days['product_cart'] / conversion_days['login']
conversion_days['purc_conv'] = conversion_days['purchase'] / conversion_days['login']
# переводим в длинный формат для построения графика
conversion_days = conversion_days.melt(id_vars=['group', 'event_dt'], value_vars=['pr_p_conv', 'pr_c_conv', 'purc_conv'])

fig = px.line(conversion_days.query('group == "A"'), 
             x='event_dt', y='value', color='event_name')

fig.update_layout(width=600, height=300, yaxis_range=[0, .8],
                 title='Конверсия в группе А по дням',
                 xaxis_title='Дата',  yaxis_title='Конверсия', yaxis_tickformat = ',.0%')
fig.show()

fig = px.line(conversion_days.query('group == "B"'), 
             x='event_dt', y='value', color='event_name')

fig.update_layout(width=600, height=300, yaxis_range=[0, .8],
                 title='Конверсия в группе В по дням',
                 xaxis_title='Дата',  yaxis_title='Конверсия', yaxis_tickformat = ',.0%')

fig.show()

Видно, что конверсия в на первой стадии - на страницу с товаром в группе А, начиная с середины рассматриваемого периода находится на уровне примерно в 65%, конверсии на страницах с корзиной и покупки в пределах 25-35% и не показывают тенденцию к снижению или увеличению. Конверсия в группе В колеблется гораздо сильнее (прежде всего вследствие значительно меньшего числа активных пользователей) и для первой стадии  находится в пределах 40-60%, а на последних двух - 20-40%. Можно заметить, что конверсия для первой стадии воронки продаж в группе А стабильно выше, чем в группе В за весь период, про следующие стадии сказать это сложнее ввиду сильных колебаний. Также не видно влияния на конверсию маркетинговой активности. 

<a id="ABtest"></a> 
## Оценка результатов A/B-тестирования

In [94]:
# посчитаем воронки событий
# для начала зададим логику вопронки
order = {'login': 0, 'product_page': 1, 'product_cart': 2, 'purchase': 3}

# посчитаем воронки по обеим группам
funnelA = (data.query('group == "A"')
           .groupby('event_name')['user_id'].nunique().reset_index()
           .sort_values(by=['event_name'], key=lambda x: x.map(order)))  # сортируем воронку в соотвествии с событиями
     
funnelB = (data.query('group == "B"')
           .groupby('event_name')['user_id'].nunique().reset_index()
           .sort_values(by=['event_name'], key=lambda x: x.map(order)))    

# и визуализируем их
fig = make_subplots(rows=1, cols=2, subplot_titles=['Группа А', 'Группа В'], shared_yaxes=True)

fig.append_trace(go.Funnel(
    x = funnelA['user_id'], y = funnelA['event_name'],
    textposition = "inside",
    textinfo = "value+percent initial") , row=1, col=1)

fig.append_trace(go.Funnel(
    x = funnelB['user_id'], y = funnelB['event_name'],
    textposition = "inside", 
    textinfo = "value+percent initial"), row=1, col=2)

fig.update_layout(height=400, width=800, title_text="Воронки продаж", showlegend=False)
fig.show()

Мы можем видеть, что ни одна конверсия в группе В не превышает аналогичную из группы А, за исключением конверсии из просмотра страницы к странице с корзиной - 49.9% в группе В против 46.5% в группе А. При этом увеличение все равно не дотягивает до требуемых 10%, а конверсия к странице с логином в группе В все равно ниже.

In [95]:
# функция считает стат значимость различия конверсии в опеределенное событие между группами по методу z-value
def p_value_btw_groups(event):
    succ1 = funnelA.set_index('event_name').loc[event, 'user_id']      # перешедшие пользователи группы А
    observ1 = funnelA.set_index('event_name').loc['login', 'user_id']  # все пользователи группы А
    succ2 = funnelB.set_index('event_name').loc[event, 'user_id']      # перешедшие пользователи группы В
    observ2 = funnelB.set_index('event_name').loc['login', 'user_id']  # все пользователи группы В

    p1 = succ1 / observ1    # доля успеха первой группы
    p2 = succ2 / observ2    # доля успеха второй группы
    p_comb = (succ1 + succ2) / (observ1 + observ2)  # доля успеха общая

    z_value = (p1 - p2) / mth.sqrt(p_comb * (1 - p_comb) * (1/observ1 + 1/observ2))
    distr = st.norm(0, 1)     # задаем нормальное распределение (0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2  # рассматриваем двустороннюю гипотезу
    return p_value

Теперь можно посмотреть, насколько статистически значима разница в различии конверсии. В качестве нулевой гипотезы примем, что конверсии на каждом из этапов в группах А и В равны, а альтернативной - что они отличаются. Проведем проверку стат-значимости с помощью z-критерия.

Примем уровень критической значимости в размере 0.05. Так как в ходе исследования мы будем проводить сравнение по 3 показателям, то с целью корректировки множественного исследования применим поправку Бонферрони, и с его учетом скорректированный уровень критической значимости составит **0.017**.

In [96]:
print('p-value о стат значимости разницы конверсии в экран товаров (product_page):',
          '{:.5f}'.format(p_value_btw_groups('product_page')))
print('\np-value о стат значимости разницы конверсии в экран корзины товаров (product_cart):',
          '{:.3f}'.format(p_value_btw_groups('product_cart')))
print('\np-value о стат значимости разницы конверсии в покупки (purchase):',
          '{:.3f}'.format(p_value_btw_groups('purchase')))

p-value о стат значимости разницы конверсии в экран товаров (product_page): 0.00001

p-value о стат значимости разницы конверсии в экран корзины товаров (product_cart): 0.225

p-value о стат значимости разницы конверсии в покупки (purchase): 0.044


Полученные значения p-value не превышают уровень критической значимости только для первого шага - перехода на экран товаров. Таким образом, мы можем говорить, что падение конверсии в группе В на первом шаге стат значимо. При этом мы не можем отвергать нулевую гипотезу о различии конверсий на втором и третьем шагах.

<a id="summary"></a>
## Общие выводы

**Выводы**
- При анализе данных выявлено, что из 6701 пользователя рассматриваемой тест-группы 1133 добавлены некорректно и их пришлось исключить из дальнейших рассчетов. 783 оказались в тестовой группе В в другом тесте, а 350 попали к нам не из региона EU.
- Пользователи оказались неравномерно распределены между группами - на группу А пришлось примерно в полтора раза больше людей.
- Тестовый период пересекается с рожденственским и новогодним промо, но видимого эффекта на поведение пользователей они не оказали. 
- В данных отсутсвуют события после 29.12.2020
- Других нарушений ТЗ не обнаружено
- Пользователи из группы А в целом ведут себя значительно активнее чем из группы В. Пользователи, дошедшие до каждого этапа из группы А совершают примерно 3 действия на любом этапе, а из В - только 2.5. 
- Мы можем наблюдать сильный всплеск активности с середины декабря и пиком 21.12. причем в группе А этот всплеск значительно выше. В первые несколько дней эксперимента активности групп были примерно одинаковыми (с учетом разного количества пользователей в группах), но потом группа В стала все больше отставать и, начиная с 14 декабря, пользователи группы В совершали в 4-5 раз меньше активных действий чем в группе А.
- Динамика конверсий не показывала тенденций к росту или пдаению в течение всего рассматриваемого периода 
- Результаты АВ тестирования показали падение конверсии в группе В по сравнению с группой А на всех этапах воронки: так, на переходе к экрану товаров она упала с 65% до 56%, на этапе страницы с корзиной - с 30% до 28%, а на этапе покупки - с 32% до 28%. Ввиду слабой активности клиентов из группы В, z-тест не показал статистическую значимость различий конверсий по второму и третьему этапу, но, что касается перехода на страницу с товарами, мы можем говорить о стат значимости падения.

**Заключение**
- АВ тест был проведен с достаточно большим количеством ошибок, но по имеющимся данным мы все же можем говорить о его отрицательном результате. Тест предлагается прекратить и признать неудачным.

# Анализ крупного сервиса для чтения книг по подписке

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [97]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [98]:
# Посчитайте, сколько книг вышло после 1 января 2000 года;
query = '''
SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,821


Всего в базе 821 книга, изданная после 01.01.2000г

In [99]:
# Для каждой книги посчитайте количество обзоров и среднюю оценку;
query = '''
WITH 
b_r AS
    (SELECT book_id, ROUND(AVG(rating), 2) AS avg_rating
    FROM ratings 
    GROUP BY book_id),
revs AS
    (SELECT book_id, COUNT(review_id) AS reviews_number
    FROM reviews 
    GROUP BY book_id)
SELECT book_id, title, avg_rating, 
    CASE WHEN reviews_number IS NULL THEN 0 ELSE reviews_number END
FROM books 
LEFT JOIN b_r USING(book_id)
LEFT JOIN revs USING(book_id)
ORDER BY book_id
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,avg_rating,reviews_number
0,1,'Salem's Lot,3.67,2
1,2,1 000 Places to See Before You Die,2.50,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.67,3
3,4,1491: New Revelations of the Americas Before C...,4.50,2
4,5,1776,4.00,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.67,3
996,997,Xenocide (Ender's Saga #3),3.40,3
997,998,Year of Wonders,3.20,4
998,999,You Suck (A Love Story #2),4.50,2


In [100]:
# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — 
# так вы исключите из анализа брошюры;
query = '''
SELECT publisher, COUNT(book_id) AS books
FROM publishers
LEFT JOIN books USING(publisher_id)
WHERE num_pages > 50
GROUP BY publisher
ORDER BY books DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,books
0,Penguin Books,42


Самое плодовитое издательство - Penguin Books c 42 книгами

In [101]:
# Определите автора с самой высокой средней оценкой книг — 
# учитывайте только книги с 50 и более оценками;
query = '''
WITH 
b_r AS
    (SELECT book_id, AVG(rating) AS avg_rt
    FROM ratings 
    GROUP BY book_id
    HAVING COUNT(*) >= 50)
SELECT author, ROUND(AVG(avg_rt), 2), COUNT(*) AS books
FROM books 
JOIN b_r USING(book_id)
JOIN authors USING(author_id)
GROUP BY author
ORDER BY AVG(avg_rt) DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,author,round,books
0,J.K. Rowling/Mary GrandPré,4.28,4


Самые высокооцениваемые авторы - J.K. Rowling в тандеме с Mary GrandPré, у них 4 книги со средним рейтингом 4.28 баллов.

In [102]:
# Посчитайте среднее количество обзоров от пользователей, которые 
# поставили больше 50 оценок.
query = '''
SELECT COUNT(*) / COUNT(DISTINCT username) AS avg_reviews
FROM reviews
WHERE username IN
    (SELECT username
    FROM ratings 
    GROUP BY username
    HAVING COUNT(*) > 50)
'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24


Активные в оценках пользователи активны и в написании обзоров - в среднем каждый такой пользователь пишет по 24 обзора на книги